In [1]:
import numpy
import pandas as pd
import numpy as np
import scipy.spatial
import tensorflow as tf 
import sklearn.naive_bayes

# Load dataset

In [2]:
train_tweets = pd.read_csv('/Users/juliachen/Desktop/DSCI552/hw/hw5/data/ps5_tweets_text.csv')
train_label = pd.read_csv('/Users/juliachen/Desktop/DSCI552/hw/hw5/data/ps5_tweets_labels_as_numbers.csv')

test_tweets = pd.read_csv('/Users/juliachen/Desktop/DSCI552/hw/hw5/data/ps5_tweets_text_for_the_kaggle_competition.csv')

In [3]:
print(f"Train data shape {train_tweets.shape}")
print(f"Test data shape {test_tweets.shape}")

Train data shape (37041, 2)
Test data shape (3798, 2)


In [4]:
#'''delete id column'''
train_tweets.drop("Id",axis=1,inplace=True)
train_label.drop("Id",axis=1,inplace=True)
#test_tweets.drop("Id",axis=1,inplace=True)

'''Combine train_tweets and train_label into train data set'''
dataset = pd.concat([train_label, train_tweets], axis=1)

In [5]:
dataset['Label'].value_counts()

3    10282
1     8930
2     6930
4     5953
0     4946
Name: Label, dtype: int64

Extremely Negative (0), Negative (1), Neutral (2), Positive (3), Extremely Positive (4)
Therefore, we can see the there the most large tweets are extremely postive, only few are extremely negative. 

In [6]:
'''Split into training and validation set'''
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
x_train, x_val, y_train, y_val = train_test_split(dataset['Tweet'], dataset['Label'], 
                                                  test_size=0.2, random_state=42)


y_val_use_for_metric = y_val
y_train = pd.get_dummies(y_train)
y_val = pd.get_dummies(y_val)

print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(29632,)
(7409,)
(29632, 5)
(7409, 5)


# Token

The process of tokenization is to represent words in a way that computer can understand and be able to train it. 

num_words: the maximum number of words to keep, based on word frequency.

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>',
                     filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                     lower=True)

#training_sequences = tokenizer.fit_on_sequences(x_train)
#valid_sequences = tokenizer.fit_on_sequences(x_val)

tokenizer.fit_on_texts(x_train)
word_ind=tokenizer.word_index

training_sentences = tokenizer.texts_to_sequences(x_train)
valid_sentences = tokenizer.texts_to_sequences(x_val)
test_sentences = tokenizer.texts_to_sequences(test_tweets['Tweet'])

#"""Pad and trunckate the squence """
traning_padded = pad_sequences(training_sentences)
valid_padded = pad_sequences(valid_sentences)
test_padded =  pad_sequences(test_sentences)

# RNN model 1

In [8]:
vocab_size = 10000
embedding_dim=16

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, name="embedding"),
    tf.keras.layers.GlobalAveragePooling1D(),
    #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
    tf.keras.layers.Dense(embedding_dim, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

model.summary()
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 85        
Total params: 160,357
Trainable params: 160,357
Non-trainable params: 0
_________________________________________________________________


In [9]:
traning_padded.shape

(29632, 70)

In [10]:
y_train.shape

(29632, 5)

In [11]:
traning_padded = np.asarray(traning_padded).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)

In [12]:
model.fit(traning_padded, y_train, epochs=10, validation_data = (valid_padded, y_val))

Epoch 1/10
926/926 [==============================] - 3s 2ms/step - loss: 1.5607 - accuracy: 0.2827 - val_loss: 1.4310 - val_accuracy: 0.3766
Epoch 2/10
926/926 [==============================] - 2s 2ms/step - loss: 1.3392 - accuracy: 0.4396 - val_loss: 1.1531 - val_accuracy: 0.5344
Epoch 3/10
926/926 [==============================] - 2s 2ms/step - loss: 1.0614 - accuracy: 0.5855 - val_loss: 1.0161 - val_accuracy: 0.6114
Epoch 4/10
926/926 [==============================] - 2s 2ms/step - loss: 0.8918 - accuracy: 0.6722 - val_loss: 0.9444 - val_accuracy: 0.6475
Epoch 5/10
926/926 [==============================] - 2s 3ms/step - loss: 0.7992 - accuracy: 0.7183 - val_loss: 0.9117 - val_accuracy: 0.6670
Epoch 6/10
926/926 [==============================] - 2s 2ms/step - loss: 0.7281 - accuracy: 0.7520 - val_loss: 0.9146 - val_accuracy: 0.6746
Epoch 7/10
926/926 [==============================] - 2s 2ms/step - loss: 0.6866 - accuracy: 0.7714 - val_loss: 0.9076 - val_accuracy: 0.6869
Epoch 

In [13]:
from sklearn import metrics
pred_val1 = model.predict(valid_padded)
pred_val1 = np.argmax(pred_val1, axis=1)

print('Accuracy: ',metrics.accuracy_score(pred_val1, y_val_use_for_metric))
print(metrics.classification_report(pred_val1, y_val_use_for_metric))

Accuracy:  0.690781481981374
              precision    recall  f1-score   support

           0       0.65      0.78      0.71       818
           1       0.64      0.64      0.64      1779
           2       0.70      0.74      0.72      1350
           3       0.68      0.64      0.66      2165
           4       0.80      0.74      0.77      1297

    accuracy                           0.69      7409
   macro avg       0.69      0.71      0.70      7409
weighted avg       0.69      0.69      0.69      7409



In [30]:
from sklearn.metrics import f1_score
print("Micro F1 score: ",f1_score(pred_val1, y_val_use_for_metric, average='micro'))
print("Macro F1 score: ",f1_score(pred_val1, y_val_use_for_metric, average='macro'))

Micro F1 score:  0.690781481981374
Macro F1 score:  0.6994808001859135


# ----------------------------------

In [14]:
pred_test1 = model.predict(test_padded)
pred_test1 = np.argmax(pred_test1, axis=1)

In [15]:
df = pd.DataFrame.from_dict({'Id': range(0,3798), 'Predicted': pred_test1})
# df.to_csv('/Users/juliachen/Desktop/rnn1', index=False)
print("Kaggle Result:", 0.64091)

Kaggle Result: 0.64091


# RNN model 2

In [16]:
vocab_size = 10000
embedding_dim=64

model2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, name="embedding"),
    #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
    tf.keras.layers.LSTM(embedding_dim),
    tf.keras.layers.Dense(embedding_dim, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

model2.summary()
model2.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          640000    
_________________________________________________________________
lstm (LSTM)                  (None, 64)                33024     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 325       
Total params: 677,509
Trainable params: 677,509
Non-trainable params: 0
_________________________________________________________________


In [17]:
model2.fit(traning_padded, y_train, epochs=10, validation_data = (valid_padded, y_val))

Epoch 1/10
926/926 [==============================] - 28s 28ms/step - loss: 1.3913 - accuracy: 0.3750 - val_loss: 0.9212 - val_accuracy: 0.6670
Epoch 2/10
926/926 [==============================] - 28s 30ms/step - loss: 0.7259 - accuracy: 0.7443 - val_loss: 0.7730 - val_accuracy: 0.7340
Epoch 3/10
926/926 [==============================] - 27s 29ms/step - loss: 0.6049 - accuracy: 0.7986 - val_loss: 0.7442 - val_accuracy: 0.7476
Epoch 4/10
926/926 [==============================] - 28s 30ms/step - loss: 0.5274 - accuracy: 0.8285 - val_loss: 0.7328 - val_accuracy: 0.7469
Epoch 5/10
926/926 [==============================] - 28s 31ms/step - loss: 0.4593 - accuracy: 0.8478 - val_loss: 0.7412 - val_accuracy: 0.7456
Epoch 6/10
926/926 [==============================] - 26s 28ms/step - loss: 0.3949 - accuracy: 0.8707 - val_loss: 0.8088 - val_accuracy: 0.7414
Epoch 7/10
926/926 [==============================] - 28s 30ms/step - loss: 0.3456 - accuracy: 0.8866 - val_loss: 0.8486 - val_accuracy:

In [26]:
pred_val2 = model2.predict(valid_padded)
pred_val2 = np.argmax(pred_val2, axis=1)

print('Accuracy: ',metrics.accuracy_score(pred_val2, y_val_use_for_metric))
print(metrics.classification_report(pred_val2, y_val_use_for_metric))

Accuracy:  0.7233094884599811
              precision    recall  f1-score   support

           0       0.72      0.72      0.72       976
           1       0.68      0.68      0.68      1754
           2       0.78      0.81      0.80      1373
           3       0.70      0.68      0.69      2095
           4       0.76      0.75      0.75      1211

    accuracy                           0.72      7409
   macro avg       0.73      0.73      0.73      7409
weighted avg       0.72      0.72      0.72      7409



In [29]:
from sklearn.metrics import f1_score
print("Micro F1 score: ",f1_score(pred_val2, y_val_use_for_metric, average='micro'))
print("Macro F1 score: ",f1_score(pred_val2, y_val_use_for_metric, average='macro'))

Micro F1 score:  0.7233094884599811
Macro F1 score:  0.729318420998671


# -----------------------------------------

In [27]:
pred_test2 = model.predict(test_padded)
pred_test2 = np.argmax(pred_test2, axis=1)

In [28]:
df2 = pd.DataFrame.from_dict({'Id': range(0,3798), 'Predicted': pred_test2})
df.to_csv('/Users/juliachen/Desktop/rnn2', index=False)
print("Kaggle Result:", 0.64091)

Kaggle Result: 0.64091
